In [1]:
import zipfile
from zipfile import ZipFile
import pandas as pd
import numpy as np
import netCDF4 as nc
from matplotlib import pyplot as plt
import xarray as xr
import os
from calendar import isleap
import warnings
warnings.filterwarnings("ignore")
import copy
import xarray as xr
from rasterio import Affine
from rasterstats import zonal_stats
from osgeo import gdal
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
import os
from shapely.ops import unary_union


In [2]:
import geopandas as gpd

path_poly = r"C:\Users\gokul\Downloads\QGIS Natural Earth\Toronto SHP\GTA.shp"
gdf = gpd.read_file(path_poly)

# Change the CRS to 4326 for the GTA shapefile. This only needs to be done once.
gdf = gdf.to_crs(epsg=4326)
gdf.to_file(path_poly)

In [3]:
path_poly = r"C:\Users\gokul\Downloads\QGIS Natural Earth\Toronto SHP\GTA.shp"
merged = gdf.geometry.unary_union
gdf_merged = gpd.GeoDataFrame(geometry=[merged])
gdf_merged = gdf_merged.set_crs('EPSG:4326')
gdf_m = gdf_merged.to_crs('EPSG:3857')
gdf_m['area_m2'] = gdf_m.geometry.area
areaofshp = gdf_m['area_m2']

In [4]:
# Define file paths
path = r'D:\Mosq Proj Data\SM Data and code\SM DATA'
basedir = path
path_poly = r"C:\Users\gokul\Downloads\QGIS Natural Earth\Toronto SHP\GTA.shp"
gdf_merged = gpd.read_file(path_poly)
geom = gdf_merged.geometry.values[0]

average = {}
filepathname = []
count = []

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))
for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    # trying to reset the image from 360 to -180 and 180 so i can use it with the shp (I believe this dataset didn't need it, so it does nothing here)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)

    # Calculate geotransform for raster data
    gt = list(gdobj.GetGeoTransform()) 
    gt[1] = np.mean(np.diff(ds['lon']))
    gt[0] = -180 - (gt[1]/2)
    gt[5] = np.mean(np.diff(ds['lat']))
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
    var = 'sm' 

    # Loop through each time point in the dataset
    for t in ds.time.data:
        array = np.squeeze(ds[var].values)
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']

        # Handle low pixel count scenarios
        if zstats[0]['count'] < 20:
            f_overlap = np.zeros(mini.shape)
            for i in range(mini.shape[0]):
                for j in range(mini.shape[1]):
                    if mini.mask[i, j]:
                        continue
                    # Calculate pixel geometry and overlay with polygon
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    pixel = Polygon([ul, (ul[0] + ma[0], ul[1]), (ul[0], ul[1] + ma[4]), (ul[0] - ma[0], ul[1]), ul])
                    f_overlap[i, j] = geom.intersection(pixel).area / pixel.area
            
            # Compute weighted average
            mini = np.where(mini < 0, 0, mini)
            mini = np.where(mini == 0, np.nan, mini)
            average[t] = np.ma.average(mini, weights=f_overlap)
            count.append(np.count_nonzero(~np.isnan(mini)))
        else:
            mini = np.where(mini < 0, 0, mini)
            mini = np.where(mini == 0, np.nan, mini)
            average[t] = np.nanmean(mini)
            count.append(np.count_nonzero(~np.isnan(mini)))

# insert the results into a dataframe and remove nan
Soil_Moisture_Data_Compiled=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Soil_Moisture_Data_Compiled['SM']=Data
Soil_Moisture_Data_Compiled['Date']=Date
Soil_Moisture_Data_Compiled['Count']=count

Soil_Moisture_Data_Compiled=Soil_Moisture_Data_Compiled.dropna()

In [5]:
Soil_Moisture_Data_Compiled

,SM,Date,Count


In [6]:
Soil_Moisture_Data_Compiled=Soil_Moisture_Data_Compiled.reset_index()
Soil_Moisture_Data_Compiled=Soil_Moisture_Data_Compiled.drop(columns=['index'])

In [7]:
Soil_Moisture_Data_Compiled_Copy=copy.deepcopy(Soil_Moisture_Data_Compiled)

In [8]:
# Getting some statistics for the data we extracted, see if the counts and whatnot are good to proceed or if we need to rerun.
Soil_Moisture_Data_Compiled_Copy['Date'] = pd.to_datetime(Soil_Moisture_Data_Compiled_Copy['Date'])

# Monthly breakdown - mean, min, max and the sum for counts
Soil_Moisture_Data_Compiled_Copy['Month'] = Soil_Moisture_Data_Compiled_Copy['Date'].dt.month
sm_count_stats_by_month = Soil_Moisture_Data_Compiled_Copy.groupby('Month')['Count'].agg(['mean', 'min', 'max'])

# Seeing the count breakdown for each month. Do some months have more count than other months?
sm_total_counts_by_month = Soil_Moisture_Data_Compiled_Copy.groupby('Month')['Count'].sum()
sm_percent_counts_by_month = (sm_total_counts_by_month / sm_total_counts_by_month.sum()) * 100

# How many counts are below average vs minimium
below_average_by_month = Soil_Moisture_Data_Compiled_Copy.groupby('Month')['Count'].apply(lambda x: (x < x.mean()).sum() / len(x) * 100)
at_minimum_by_month = Soil_Moisture_Data_Compiled_Copy.groupby('Month')['Count'].apply(lambda x: (x == x.min()).sum() / len(x) * 100)
sm_count_stats_by_month = sm_count_stats_by_month.merge(sm_percent_counts_by_month, on='Month')
sm_count_stats_by_month = sm_count_stats_by_month.merge(below_average_by_month, on='Month')
sm_count_stats_by_month = sm_count_stats_by_month.merge(at_minimum_by_month, on='Month')
sm_count_stats_by_month.rename(columns={'mean': 'Mean Count', 'min': 'Min Count', 'max': 'Max Count', 'Count_x': 'Percentage of Total Counts', 'Count_y': 'Percentage of Data Below Average Count', 'Count': 'Percentage of Data at Minimum Count'}, inplace=True)
sm_count_stats_by_month = sm_count_stats_by_month.drop('Percentage of Total Counts', axis=1)
total_data_by_month = Soil_Moisture_Data_Compiled_Copy.groupby('Month')['Count'].count()
percent_data_by_month = (total_data_by_month / total_data_by_month.sum()) * 100
sm_count_stats_by_month = sm_count_stats_by_month.merge(percent_data_by_month, on='Month')
sm_count_stats_by_month.rename(columns={'Count': 'Percentage of Total Data'}, inplace=True)

sm_count_stats_by_month = sm_count_stats_by_month.reset_index()

# print('\nCount Statistics by Month SM:')
# print(sm_count_stats_by_month)

In [9]:
# filter data so min data has at least 4 counts
SM=Soil_Moisture_Data_Compiled[Soil_Moisture_Data_Compiled['Count']>4]
SM_Copy=copy.deepcopy(SM)
SM_Copy['Date'] = pd.to_datetime(SM_Copy['Date'])
SM_Copy['Month'] = SM_Copy['Date'].dt.month
sm_count_stats_by_month = SM_Copy.groupby('Month')['Count'].agg(['mean', 'min', 'max'])
sm_total_counts_by_month = SM_Copy.groupby('Month')['Count'].sum()
sm_percent_counts_by_month = (sm_total_counts_by_month / sm_total_counts_by_month.sum()) * 100
below_average_by_month = SM_Copy.groupby('Month')['Count'].apply(lambda x: (x < x.mean()).sum() / len(x) * 100)
at_minimum_by_month = SM_Copy.groupby('Month')['Count'].apply(lambda x: (x == x.min()).sum() / len(x) * 100)
sm_count_stats_by_month = sm_count_stats_by_month.merge(sm_percent_counts_by_month, on='Month')
sm_count_stats_by_month = sm_count_stats_by_month.merge(below_average_by_month, on='Month')
sm_count_stats_by_month = sm_count_stats_by_month.merge(at_minimum_by_month, on='Month')
sm_count_stats_by_month.rename(columns={'mean': 'Mean Count', 'min': 'Min Count', 'max': 'Max Count', 'Count_x': 'Percentage of Total Counts', 'Count_y': 'Percentage of Data Below Average Count', 'Count': 'Percentage of Data at Minimum Count'}, inplace=True)
sm_count_stats_by_month = sm_count_stats_by_month.drop('Percentage of Total Counts', axis=1)

total_data_by_month = SM_Copy.groupby('Month')['Count'].count()
percent_data_by_month = (total_data_by_month / total_data_by_month.sum()) * 100
sm_count_stats_by_month = sm_count_stats_by_month.merge(percent_data_by_month, on='Month')
sm_count_stats_by_month.rename(columns={'Count': 'Percentage of Total Data'}, inplace=True)

sm_count_stats_by_month = sm_count_stats_by_month.reset_index()

print('\nCount Statistics by Month SM:')
print(sm_count_stats_by_month)


Count Statistics by Month SM:
Empty DataFrame
Columns: [Month, Mean Count, Min Count, Max Count, Percentage of Data Below Average Count, Percentage of Data at Minimum Count, Percentage of Total Data]
Index: []


In [10]:
# SM.to_excel(r'Path\To\File\SoilMoisturePolygon.xlsx')